### Для чего нужен PyTorch?

1. автоматическое дифференцирование
2. работа с датасетами
3. построение моделей
4. обучение моделей

In [52]:
import torch

### Данные

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class PhotosDataset(Dataset):
    
    def __init__(self, images_dir, target_dir, transforms=None):
        self.img_paths = []
        self.target_paths = []
        """
        заполняем пути к файлам
        """
        self.transforms = transforms
        
    def __len__(self):
        return len(self.img_paths)
                   
    def __getitem__(self, idx):
        """
        1. load data object (image, text, graphs, ...)
        2. transform data (self.transforms(...))
        """
        img_name = self.img_paths[idx]
        target_name = self.target_paths[idx]
        sample = (
            np.asarray(Image.open(img_name)),
            np.asarray(Image.open(target_name)),
        )
        sample = self.transforms(sample)
        
        return sample

In [ ]:
train_dataset = PhotosDataset(
    images_dir=DATA_PATH + 'train/',
    target_dir=DATA_PATH + 'train_mask/',
    transforms= img_tr
)
test_dataset = PhotosDataset(
    images_dir=DATA_PATH + 'test/',
    target_dir=DATA_PATH + 'test_mask/',
    transforms= test_tr
)

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_data_loader = DataLoader(test_dataset, batch_size=8)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay)

In [ ]:
model.train()
from data, target in train_data_loader:
    optimizer.zero_grad()
    output = model(data)
    l = loss(output, target)
    l.backward()
    optimizer.step()
    

In [ ]:
out = []
model.eval()
with torch.no_grad():
    from data, target in test_data_loader:
        output = model(data)
        l = loss(output, target)
        out += [ l ]
    
print(sum(out) / len(out))

### Модель

In [42]:
import torch.nn as nn

In [43]:
input_sz = 15
output_sz = 2

In [46]:
model_1 = nn.Sequential(
    nn.Linear(input_sz, 10),
    nn.Dropout(0.1),
    nn.ReLU(),
    nn.Linear(10, output_sz),
    nn.Sigmoid()
)
model_1

Sequential(
  (0): Linear(in_features=15, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
  (3): Sigmoid()
)

In [ ]:
out = model_1.forward(data)
l = loss(out, target)
l.backward()

In [47]:
class Model(nn.Module):
    def __init__(self, input_sz, output_sz):
        super(Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_sz, 10),
            nn.ReLU(),
            nn.Linear(10, output_sz),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.layers(x)
    

model_2 = Model(input_sz, output_sz)
model_2

Model(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=2, bias=True)
    (3): Sigmoid()
  )
)

In [49]:
class Model(nn.Module):
    def __init__(self, input_sz, output_sz):
        super(Model, self).__init__()
        self.W_1 = nn.Parameter(torch.zeros((input_sz, 10)).uniform_(0.0,1.0))
        self.b_1 = nn.Parameter(torch.zeros((10)).uniform_(0.0,1.0))
        self.layers = nn.ReLU()
        self.W_2 = nn.Parameter(torch.zeros((10, output_sz)).uniform_(0.0,1.0))
        self.b_2 = nn.Parameter(torch.zeros((output_sz)).uniform_(0.0,1.0))
        self.sigm = nn.Sigmoid()
        self.show = True
    
    def forward(self, x):
        x_1 = x @ self.W_1 + self.b_1
        x_2 = self.layers(x_1)
        x_3 = torch.matmul(x_2, self.W_2) + self.b_2
        return self.sigm(x_3)

model_3 = Model(input_sz, output_sz)
model_3

Model(
  (layers): ReLU()
  (sigm): Sigmoid()
)

In [ ]:
torch.save(model.state_dict(), "model.pth")

In [ ]:
model = Model(input_sz, output_sz)
model.load_state_dict(torch.load("model.pth"))